In [2]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt


In [3]:
data=sio.loadmat('ex4data1')

In [4]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [5]:
x=data['X']
y=data['y']

In [6]:
x=np.insert(x,0,1,axis=1)

In [7]:
x.shape,y.shape

((5000, 401), (5000, 1))

In [8]:
#进行hot one编码处理

In [9]:
def hot_one(y):
    result=[]
    for i in y:
        a=np.zeros(10)
        a[i-1]=1
        result.append(a)
    return np.array(result)

In [10]:
y=hot_one(y)

In [11]:
theta=sio.loadmat('ex4weights')

In [12]:
theta1=theta['Theta1']
theta2=theta['Theta2']
theta1.shape,theta2.shape

((25, 401), (10, 26))

In [13]:
def serialize(a,b):
    return np.append(a.flatten(),b.flatten())

In [14]:
thetavec=serialize(theta1,theta2)

In [15]:
thetavec.shape

(10285,)

In [16]:
def deserialize(thetavec):
    theta1=thetavec[:25*401].reshape(25,401)
    theta2=thetavec[25*401:].reshape(10,26)
    return theta1,theta2

In [17]:
theta1,theta2=deserialize(thetavec)

In [18]:
theta1.shape,theta2.shape

((25, 401), (10, 26))

In [19]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [20]:
def forward(thetavec,x):
    theta1,theta2=deserialize(thetavec)
    a1=x
    z2=a1@theta1.T
    a2=sigmoid(z2)
    a2=np.insert(a2,0,1,axis=1)
    z3=a2@theta2.T
    h=sigmoid(z3)
    return a1,z2,a2,z3,h

In [21]:
#不带正则化

In [22]:
def cost(thetavec,x,y):
    a1,z2,a2,z3,h=forward(thetavec,x)
    j=-np.sum(y*np.log(h)+(1-y)*np.log(1-h))/len(x)
    return j

In [23]:
cost(thetavec,x,y)

0.2876291651613189

In [24]:
#带正则化项的误差函数
def reg_cost(thetavec,x,y,labda):
    theta1,theta2=deserialize(thetavec)
    sum1=np.sum(np.power(theta1[:,1:],2))
    sum2=np.sum(np.power(theta2[:,1:],2))
    reg=(sum1+sum2)*labda/(2*len(x))
    return reg+cost(thetavec,x,y)

In [25]:
labda=1
reg_cost(thetavec,x,y,labda)

0.38376985909092365

In [26]:
def sigmoidgrident(z):
    return sigmoid(z)*(1-sigmoid(z))

In [27]:
def grident(thetavec,x,y):
    theta1,theta2=deserialize(thetavec)
    a1,z2,a2,z3,h=forward(thetavec,x)
    d3=h-y
    d2=d3@theta2[:,1:]*sigmoidgrident(z2)
    D2=(d3.T@a2)/len(x)
    D1=(d2.T@a1)/len(x)
    return  serialize(D1,D2)

In [28]:
def reg_grident(thetavec,x,y,labda):
    D=grident(thetavec,x,y)
    D1,D2=deserialize(D)
    theta1,theta2=deserialize(thetavec)
    D1[:,1:]=D1[:,1:]+theta1[:,1:]*labda/len(x)
    D2[:,1:]=D2[:,1:]+theta2[:,1:]*labda/len(x)
    return serialize(D1,D2)

In [29]:
from scipy.optimize import minimize
def training(x,y):
    ini_theta=np.random.uniform(-0.5,0.5,10285)
    res=minimize(reg_cost,ini_theta,args=(x,y,labda),method='TNC',jac=reg_grident,options={'maxiter':300})
    return res

In [30]:
labda=10
res=training(x,y)

In [31]:
y_raw=data['y'].reshape(5000,)

In [32]:
a1,z2,a2,z3,h=forward(res.x,x)

In [33]:
y_pre=np.argmax(h,axis=1)+1
acc=np.mean(y_pre==y_raw)

In [34]:
acc

0.9412

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [37]:
data['y'].shape

(5000, 1)